In [ ]:
from googleapiclient.discovery import build
from youtube_transcript_api import YouTubeTranscriptApi
import json

# Replace with your API key
API_KEY = "YOUR_YOUTUBE_API_KEY"

def get_video_id(youtube_url):
    """Extracts video ID from a YouTube URL."""
    if "watch?v=" in youtube_url:
        return youtube_url.split("watch?v=")[-1].split("&")[0]
    elif "youtu.be/" in youtube_url:
        return youtube_url.split("youtu.be/")[-1].split("?")[0]
    else:
        raise ValueError("Invalid YouTube URL")

def get_caption_tracks(video_id):
    """Fetches available caption tracks for a video."""
    youtube = build("youtube", "v3", developerKey=API_KEY)
    
    response = youtube.captions().list(
        part="snippet",
        videoId=video_id
    ).execute()
    
    captions = [
        {
            "id": item["id"],
            "language": item["snippet"]["language"],
            "name": item["snippet"].get("name", "Unknown"),
        }
        for item in response.get("items", [])
    ]
    
    return captions

def get_captions_json(video_id, lang="en"):
    """Retrieves closed captions for the video in JSON format."""
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=[lang])
        return json.dumps(transcript, indent=2)
    except Exception as e:
        return json.dumps({"error": str(e)})

if __name__ == "__main__":
    youtube_url = "https://www.youtube.com/watch?v=VIDEO_ID"
    video_id = get_video_id(youtube_url)
    
    print("Available Captions:", get_caption_tracks(video_id))
    print("Captions JSON:", get_captions_json(video_id, lang="en"))